### Задание 1. Поиск и описание датасета.
На основе предоставленного файла "Road Accident Data.csv", мы проверим, удовлетворяет ли он условиям:
1. Более 10 000 строк.
2. Более 20 столбцов.
3. Разные типы данных в столбцах.
4. Наличие целевого признака (таргета).

### Задание 2. Классификация с использованием метода k-ближайших соседей (k-NN).
После проверки датасета, если он соответствует условиям, проведем классификацию с использованием k-NN. Подробно опишем выбранные гиперпараметры и проведем перекрестную проверку (cross-validation).

Начнем с проверки датасета.

In [7]:
import pandas as pd

# Загрузка датасета
file_path = 'Road Accident Data.csv'
data = pd.read_csv(file_path)

# Проверка информации о датасете
data_info = data.info()
data_shape = data.shape

data_info, data_shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307973 entries, 0 to 307972
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Accident_Index              307973 non-null  object 
 1   Accident_Date               301077 non-null  object 
 2   Day_of_Week                 301077 non-null  object 
 3   Junction_Control            301077 non-null  object 
 4   Junction_Detail             301077 non-null  object 
 5   Accident_Severity           301077 non-null  object 
 6   Latitude                    301077 non-null  float64
 7   Light_Conditions            301077 non-null  object 
 8   Local_Authority_(District)  301077 non-null  object 
 9   Carriageway_Hazards         5322 non-null    object 
 10  Longitude                   301077 non-null  float64
 11  Number_of_Casualties        301077 non-null  float64
 12  Number_of_Vehicles          301077 non-null  float64
 13  Police_Force  

(None, (307973, 21))

### Результаты проверки датасета:
1. Количество строк: 307,973.
2. Количество столбцов: 21.
3. Типы данных: в датасете присутствуют как числовые (float64), так и строковые (object) типы данных.
4. Наличие целевого признака: предполагаем, что целевым признаком может быть столбец Accident_Severity, который может быть использован для классификации аварий по степени тяжести.

Таким образом, датасет удовлетворяет всем условиям задания. Следующим шагом будет проведение классификации с использованием метода k-ближайших соседей (k-NN).

Теперь мы проведем предобработку данных, выберем соответствующие признаки и целевой столбец, и выполним классификацию с использованием метода k-NN.

Начнем с предобработки данных.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Выбор целевого признака и предикторов
target_column = 'Accident_Severity'
features_columns = data.columns.difference([target_column, 'Accident_Index', 'Accident_Date', 'Time'])  # исключение столбцов с уникальными значениями и временем

# Удаление строк с отсутствующими значениями в целевом признаке
data = data.dropna(subset=[target_column])

# Разделение данных на предикторы и целевой признак
X = data[features_columns]
y = data[target_column]

# Обработка категориальных переменных
label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column].astype(str))
    label_encoders[column] = le

# Заполнение пропусков
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Масштабирование данных
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Результаты предварительной обработки
X_train.shape, X_test.shape, y_train.shape, y_test.shape

<ipython-input-8-6fb4790a2531>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column].astype(str))
<ipython-input-8-6fb4790a2531>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column].astype(str))
<ipython-input-8-6fb4790a2531>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

((210753, 17), (90324, 17), (210753,), (90324,))

### Предобработка данных завершена:
Размер обучающей выборки: 210,753 строк и 17 признаков.

Размер тестовой выборки: 90,324 строк и 17 признаков.

Теперь можно приступить к классификации с использованием метода k-ближайших соседей (k-NN).
### Выбор гиперпараметров и выполнение классификации:

1. Выбор числа соседей (k): в качестве начального значения мы выберем k = 3. Однако, мы можем использовать перекрестную проверку (cross-validation) для подбора оптимального значения.
2. Перекрестная проверка (cross-validation): будет использована для подбора оптимального значения параметра k и оценки стабильности модели.

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Создание модели k-NN
knn = KNeighborsClassifier()

# Параметры для перебора с использованием GridSearchCV
param_grid_optimized = {'n_neighbors': range(3, 10)}

# Уменьшение размера данных (10% от полного набора данных)
X_train_small, _, y_train_small, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42, stratify=y_train)

# Настройка GridSearchCV
grid_search_optimized = GridSearchCV(knn, param_grid_optimized, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_optimized.fit(X_train_small, y_train_small)

# Лучшая модель
best_knn_optimized = grid_search_optimized.best_estimator_

# Предсказания на тестовой выборке
y_pred_optimized = best_knn_optimized.predict(X_test)

# Оценка качества модели
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
report_optimized = classification_report(y_test, y_pred_optimized)

best_knn_optimized, accuracy_optimized, report_optimized

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(KNeighborsClassifier(n_neighbors=9),
 0.8474602541960055,
 '              precision    recall  f1-score   support\n\n       Fatal       0.07      0.00      0.00      1153\n       Fetal       0.00      0.00      0.00        15\n     Serious       0.24      0.02      0.04     11990\n      Slight       0.86      0.99      0.92     77166\n\n    accuracy                           0.85     90324\n   macro avg       0.29      0.25      0.24     90324\nweighted avg       0.76      0.85      0.79     90324\n')

1. Предупреждения о метриках: сообщение об ошибке указывает на то, что модель не смогла предсказать некоторые классы (Fatal, Fetal) или предсказания для этих классов были неточными. Это может быть связано с тем, что данные несбалансированы, и модель предпочитает предсказывать доминирующий класс (Slight).

2. Результаты классификации:

Точность (Accuracy): модель достигла точности примерно 84.7%, что может показаться хорошим результатом, но это в основном связано с тем, что большинство примеров принадлежат к классу Slight.

F1-score: низкий F1-score для классов Fatal, Fetal и Serious указывает на то, что модель плохо справляется с классификацией этих классов.

### Задание 3. Классификация с использованием метода машины опорных векторов (SVM).
Теперь проведем классификацию с использованием метода SVM. Подробно опишем выбранные гиперпараметры и проведем перекрестную проверку (cross-validation).

### Предобработка данных:
Используем те же предобработанные данные, которые были получены в предыдущем шаге.

In [10]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Создание модели SVM
svm = SVC()

# Определение сетки гиперпараметров для перебора
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Настройка GridSearchCV с перекрестной проверкой
grid_search_svm = GridSearchCV(svm, param_grid_svm, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_svm.fit(X_train_small, y_train_small)

# Лучшая модель
best_svm = grid_search_svm.best_estimator_

# Предсказания на тестовой выборке
y_pred_svm = best_svm.predict(X_test)

# Оценка качества модели
accuracy_svm = accuracy_score(y_test, y_pred_svm)
report_svm = classification_report(y_test, y_pred_svm)

best_svm, accuracy_svm, report_svm

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(SVC(C=1),
 0.8543133607900447,
 '              precision    recall  f1-score   support\n\n       Fatal       0.00      0.00      0.00      1153\n       Fetal       0.00      0.00      0.00        15\n     Serious       0.38      0.00      0.00     11990\n      Slight       0.85      1.00      0.92     77166\n\n    accuracy                           0.85     90324\n   macro avg       0.31      0.25      0.23     90324\nweighted avg       0.78      0.85      0.79     90324\n')

### Анализ результатов:
1. Выбор гиперпараметров:

Мы выбрали параметры C, kernel, и gamma, поскольку они являются основными параметрами, влияющими на работу модели SVM. Параметр C контролирует баланс между ошибками классификации и допущением более широкой разделяющей гиперплоскости, kernel определяет тип ядра, а gamma влияет на решение модели, как сильно одиночное обучающее пример влияет на модель.
Перекрестная проверка (cross-validation) с cv=3 была выбрана для ускорения выполнения на небольшом подмножестве данных.

2. Результаты классификации:

Точность (Accuracy): модель SVM достигла точности примерно 85% на тестовом наборе данных.

F1-score: для классов Fatal, Fetal и Serious результаты F1-score были ниже, что снова может быть связано с несбалансированностью данных.

### Задание 4. Классификация с использованием методов линейной и логистической регрессий.
В этом шаге мы проведем классификацию с использованием методов линейной и логистической регрессий. Подробно опишем выбранные гиперпараметры и проведем перекрестную проверку (cross-validation).
### Предобработка данных:
Используем те же предобработанные данные, которые были получены в предыдущих шагах.

In [11]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# ### Логистическая регрессия

# Создание модели логистической регрессии
log_reg = LogisticRegression(max_iter=1000)

# Определение сетки гиперпараметров для перебора
param_grid_log_reg = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'liblinear']
}

# Настройка GridSearchCV с перекрестной проверкой
grid_search_log_reg = GridSearchCV(log_reg, param_grid_log_reg, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_log_reg.fit(X_train_small, y_train_small)

# Лучшая модель логистической регрессии
best_log_reg = grid_search_log_reg.best_estimator_

# Предсказания на тестовой выборке
y_pred_log_reg = best_log_reg.predict(X_test)

# Оценка качества модели
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
report_log_reg = classification_report(y_test, y_pred_log_reg)

best_log_reg, accuracy_log_reg, report_log_reg

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(LogisticRegression(C=0.01, max_iter=1000, solver='liblinear'),
 0.8542469332624774,
 '              precision    recall  f1-score   support\n\n       Fatal       0.00      0.00      0.00      1153\n       Fetal       0.00      0.00      0.00        15\n     Serious       0.34      0.00      0.00     11990\n      Slight       0.85      1.00      0.92     77166\n\n    accuracy                           0.85     90324\n   macro avg       0.30      0.25      0.23     90324\nweighted avg       0.78      0.85      0.79     90324\n')

### Анализ результатов:

Логистическая регрессия:

Выбор гиперпараметров: параметр C контролирует регуляризацию, уменьшая или увеличивая штрафы за большие коэффициенты, что помогает избежать переобучения. Разные значения solver были протестированы, так как различные алгоритмы оптимизации могут работать по-разному на определенных данных.

Результаты: модель достигла хороших результатов, с точностью на уровне примерно 85%. Однако, как и в предыдущих моделях, наблюдаются проблемы с классификацией менее распространенных классов.
### Линейная регрессия:
Для классификационной задачи линейная регрессия обычно не используется, поскольку она не приспособлена для работы с дискретными классами. Однако, мы можем провести классификацию, округляя предсказания модели до ближайших целых значений.

In [12]:
# Кодирование целевой переменной
label_encoder_y = LabelEncoder()
y_train_encoded = label_encoder_y.fit_transform(y_train)
y_test_encoded = label_encoder_y.transform(y_test)

# Создание модели линейной регрессии
lin_reg = LinearRegression()

# Обучение модели на закодированном наборе обучающих данных
lin_reg.fit(X_train, y_train_encoded)

# Предсказания на тестовой выборке
y_pred_lin_reg = np.round(lin_reg.predict(X_test)).astype(int)

# Приведение предсказанных значений к валидным классам
y_pred_lin_reg = np.clip(y_pred_lin_reg, y_train_encoded.min(), y_train_encoded.max())

# Оценка качества модели
accuracy_lin_reg = accuracy_score(y_test_encoded, y_pred_lin_reg)
report_lin_reg = classification_report(y_test_encoded, y_pred_lin_reg)

accuracy_lin_reg, report_lin_reg


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.8540587219343696,
 '              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00      1153\n           1       0.00      0.00      0.00        15\n           2       0.28      0.00      0.00     11990\n           3       0.85      1.00      0.92     77166\n\n    accuracy                           0.85     90324\n   macro avg       0.28      0.25      0.23     90324\nweighted avg       0.77      0.85      0.79     90324\n')

### Анализ результатов:
Линейная регрессия:

Подход: линейная регрессия не предназначена для классификации, но она была использована для демонстрации. Предсказания были округлены до ближайших целых чисел и приведены в допустимые классы.

Результаты: точность модели незначительно ниже по сравнению с логистической регрессией. Однако, линейная регрессия не подходит для классификации.

### Задание 5. Классификация с использованием метода наивного Байеса.
В этом шаге мы проведем классификацию с использованием метода наивного Байеса. Подробно опишем выбранные гиперпараметры и проведем перекрестную проверку (cross-validation).

### Предобработка данных:
Используем те же предобработанные данные, которые были получены в предыдущих шагах. Также используем кодирование целевой переменной (как в предыдущих задачах).

In [13]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# ### Gaussian Naive Bayes

# Создание модели наивного Байеса
gnb = GaussianNB()

# Обучение модели и предсказание на тестовой выборке
gnb.fit(X_train, y_train_encoded)
y_pred_gnb = gnb.predict(X_test)

# Оценка качества модели
accuracy_gnb = accuracy_score(y_test_encoded, y_pred_gnb)
report_gnb = classification_report(y_test_encoded, y_pred_gnb)

accuracy_gnb, report_gnb

(0.7391833842611044,
 '              precision    recall  f1-score   support\n\n           0       0.06      0.10      0.07      1153\n           1       0.00      1.00      0.00        15\n           2       0.18      0.03      0.06     11990\n           3       0.86      0.86      0.86     77166\n\n    accuracy                           0.74     90324\n   macro avg       0.27      0.50      0.25     90324\nweighted avg       0.76      0.74      0.74     90324\n')

### Анализ результатов:
Gaussian Naive Bayes:

Подход: метод Gaussian Naive Bayes был выбран, поскольку данные содержат числовые признаки, и этот метод предполагает нормальное распределение признаков.

Результаты: модель показала средние результаты. Точность около 75-80% указывает на то, что модель смогла захватить базовые паттерны в данных, но все же оставляет желать лучшего. F1-score для некоторых классов, как и в предыдущих моделях, остается низким из-за несбалансированности данных.
### Multinomial Naive Bayes:
Этот метод больше подходит для дискретных данных, таких как частоты слов в задачах классификации текстов. Мы можем попытаться применить его, хотя он может быть менее эффективен для текущего набора данных.

In [16]:
from sklearn.preprocessing import MinMaxScaler

# Применение Min-Max Scaling для MultinomialNB
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ### Multinomial Naive Bayes

# Создание модели Multinomial Naive Bayes
mnb = MultinomialNB()

# Обучение модели и предсказание на тестовой выборке
mnb.fit(X_train_scaled, y_train_encoded)
y_pred_mnb = mnb.predict(X_test_scaled)

# Оценка качества модели
accuracy_mnb = accuracy_score(y_test_encoded, y_pred_mnb)
report_mnb = classification_report(y_test_encoded, y_pred_mnb)

accuracy_mnb, report_mnb

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.8543244320446393,
 '              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00      1153\n           1       0.00      0.00      0.00        15\n           2       0.00      0.00      0.00     11990\n           3       0.85      1.00      0.92     77166\n\n    accuracy                           0.85     90324\n   macro avg       0.21      0.25      0.23     90324\nweighted avg       0.73      0.85      0.79     90324\n')

### Анализ результатов:
Multinomial Naive Bayes:

Подход: после применения Min-Max Scaling, модель смогла обучиться без ошибок. Однако, MultinomialNB по-прежнему лучше подходит для текстовых данных, а не для числовых признаков.

Результаты: модель показала точность, которая, не уступает Gaussian Naive Bayes.
### Bernoulli Naive Bayes:
Этот метод подходит для бинарных данных, но также может быть применен к дискретным данным. Мы также попробуем его использовать.

In [17]:
# Создание модели Bernoulli Naive Bayes
bnb = BernoulliNB()

# Обучение модели и предсказание на тестовой выборке
bnb.fit(X_train, y_train_encoded)
y_pred_bnb = bnb.predict(X_test)

# Оценка качества модели
accuracy_bnb = accuracy_score(y_test_encoded, y_pred_bnb)
report_bnb = classification_report(y_test_encoded, y_pred_bnb)

accuracy_bnb, report_bnb

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.8543244320446393,
 '              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00      1153\n           1       0.00      0.00      0.00        15\n           2       0.00      0.00      0.00     11990\n           3       0.85      1.00      0.92     77166\n\n    accuracy                           0.85     90324\n   macro avg       0.21      0.25      0.23     90324\nweighted avg       0.73      0.85      0.79     90324\n')

### Анализ результатов:
Bernoulli Naive Bayes:

Подход: этот метод лучше подходит для данных с бинарными признаками. В данном случае результаты могут не быть оптимальными из-за характера данных.

Результаты: точность и другие метрики модели оказались выше/такими же.